<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/multi_agent_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install smolagents -q
!pip install bitsandbytes -q

In [7]:
!nvidia-smi

Mon Jan 13 18:29:30 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   76C    P0              34W /  72W |   8183MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [5]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

import torch
import re

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
from smolagents import CodeAgent, DuckDuckGoSearchTool, Tool

# 1. Load the Llama 2 7B chat model with 4-bit quantization
model_id = "meta-llama/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model_id)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", quantization_config=bnb_config)
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

# 2. Define a wrapper to make the Llama 2 model compatible with smol_agents
class Llama2Wrapper:
    def __init__(self, generator, tokenizer):
        self.generator = generator
        self.tokenizer = tokenizer

    def __call__(self, text, stop_sequences=None, **kwargs):
        # Handle different input types
        if isinstance(text, str):
            pass  # If it's a string, no need to convert
        elif isinstance(text, (list, dict)):
            # If it's a list or dictionary, join elements into a single string
            text = ' '.join([str(item) for item in text])
        else:
            text = str(text)  # Convert other types to a string

        inputs = self.tokenizer(text, return_tensors="pt")
        input_ids = inputs.input_ids.to(self.generator.model.device)

        # Generate output without stopping criteria
        sequences = self.generator.model.generate(
            input_ids=input_ids,
            max_length=512,  # Control overall length
            **kwargs
        )
        output = self.tokenizer.decode(sequences[0], skip_special_tokens=True)

        # Post-processing to mimic stopping criteria
        if stop_sequences:
            for stop_seq in stop_sequences:
                if stop_seq in output:
                    output = output[:output.index(stop_seq)]  # Truncate at stop sequence
                    break

        return {"generated_text": output}

# 3. Create the wrapped Llama 2 model
llm = Llama2Wrapper(generator, tokenizer)

# The question
question = "How many seconds would it take for a leopard at full speed to run through Pont des Arts?"

# Map answer choices to actual answers
answer_choices = {
    "A": "10 seconds",
    "B": "20 seconds",
    "C": "30 seconds",
    "D": "40 seconds",
    "E": "50 seconds"
}

# Modify the prompt and post-process the response with stricter filtering
prompt = f"""
The user asked: {question}

Even though it's unlikely a leopard would run across Pont des Arts, let's make a hypothetical estimate.
Consider the following:
* The average speed of a leopard is around 58 km/h (36 mph).
* The length of Pont des Arts is approximately 155 meters (509 feet).
* Convert the speed to meters per second: 58 km/h * (1000 m/km) * (1 h/3600 s) ≈ 16.1 m/s
* Divide the length of the bridge by the speed of the leopard: 155 m / 16.1 m/s ≈ 9.6 s

Based on these factors, estimate how many seconds it might take.
Choose the closest option:
A. 10 seconds
B. 20 seconds
C. 30 seconds
D. 40 seconds
E. 50 seconds

The closest option is: **A. 10 seconds**

Respond with only the letter corresponding to the closest option:
"""  # Stronger constraint and response format

final_answer = llm(prompt)  # Directly call the language model
final_answer_text = final_answer['generated_text'] # Assign output to final_answer_text

# Post-process the response using regular expressions
final_answer_text = re.sub(r"[^A-E]", "", final_answer_text)  # Remove anything that's not A-E
final_answer_text = final_answer_text[0]  # Extract only the first letter

# Get the actual answer from the answer choices
actual_answer = answer_choices.get(final_answer_text)

# Print the question and the answer
#print(f"Question: {question}")
#print(f"Answer: {actual_answer}") # Print the actual answer instead of just the letter


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


In [6]:
# Print the question and the answer
print(f"Question: {question}")
print(f"Answer: {actual_answer}") # Print the actual answer instead of just the letter

Question: How many seconds would it take for a leopard at full speed to run through Pont des Arts?
Answer: 10 seconds
